In [39]:

from PIL import Image

import numpy as np

import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import cv2

import torch.optim as optim


# FCN

Avui farem feina amb xarxes que no tenen cap tipus de capa _fully connected_ per tant serà una xarxa _Fully Convolutional Network_ (FCN). Quan parlam d'una xarxa FCN, ens referim a xarxes tipus VGG. Ens anirà molt bé fer aquesta pràctica per poder passar a xarxes que fan segmentació ja que la meitat d'aquestes és una FCN.

Emprarem un dataset propi per fer aquesta pràctica. Això implica fer una mica més de feina per preparar les dades. En concret emprarem una versió del conjunt de dades : AIXI_SHAPE propi d'en Miquel Miró. [Enllaç](https://uibes-my.sharepoint.com/:u:/g/personal/gma040_id_uib_eu/EcsNAK5mkXRBqayDo1JYeooBWCf1lpRA-YJHT_kDF4J_nA?e=apkCql)

La feina d'avui és "lliure" (considerau-ho una mini-pràctica), el conjunt de dades que teniu a la vostra disposició permet fer com a mínim 4 feines:

1. **Regressió**: Contar quants d'objectes hi ha
2. **Regressió de classe**: Contar quants d'objectes de cada classe hi ha en una imatge.
3. **Detecció**: Mostrar on hi ha cada un dels objectes. Es podrien emprar xarxes ja fetes per aquesta tasca (tant les que teniu disponibles a pytorch com altres que trobeu)
4. **Segmentació**: Encara no en sabem, però ho resoldrem la setmana que vé.

Avui heu de fer una de les dues primeres. Tant podeu triar fer-ho amb les imatges amb textura, com amb les imatges binaries que serveixen com a _ground truth_ (gt).

Les imatges del gt són imatges binàries (0,1) de 3 canals on a cada canal hi ha un tipus d'objecte . Per poder contar el nombre d'objectes possiblement haureu de emprar les funcions `cv2.add` per unir tots els canals en una sola imatge i la funció `cv2.findContours` per contar el nombre d'objectes en una imatge. A més podeu demanar-me ajuda a mi o al vostre amic ChatGPT.


#### [Inciso] Si emprau Colab:

Aquest codi us serveix per connectar colab amb google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%ls
%cd #TODO al vostre sistema de fitxers


## Preparació de les Dades
Per preparar el conjunt de dades necessitarem fer algunes pases:

1. Crear una llista amb les imatges 
2. Crear una classe que ens permeti obtenir una tupla (imatge, etiqueta)
3. Emprar els objectes DataLoader com hem fet sempre, aquí no trobareu cap canvi

#### Crear una llista amb les imatges 


In [83]:
import os

path_train = "aixi_shape_256_texture/train/"## TODO: posar el vostre path

files = os.listdir(path_train)
img_files = list([f"{path_train}{p}" for p in files if p.endswith('.png')])
label_files = list([f"{path_train}gt/{p}" for p in files if p.endswith('.png')])

## TODO: Comprovar que img_files i label_files tenen la informació del path de cada imatge
print(label_files)

['aixi_shape_256_texture/train/gt/00000.png', 'aixi_shape_256_texture/train/gt/00001.png', 'aixi_shape_256_texture/train/gt/00002.png', 'aixi_shape_256_texture/train/gt/00003.png', 'aixi_shape_256_texture/train/gt/00004.png', 'aixi_shape_256_texture/train/gt/00005.png', 'aixi_shape_256_texture/train/gt/00006.png', 'aixi_shape_256_texture/train/gt/00007.png', 'aixi_shape_256_texture/train/gt/00008.png', 'aixi_shape_256_texture/train/gt/00009.png', 'aixi_shape_256_texture/train/gt/00010.png', 'aixi_shape_256_texture/train/gt/00011.png', 'aixi_shape_256_texture/train/gt/00012.png', 'aixi_shape_256_texture/train/gt/00013.png', 'aixi_shape_256_texture/train/gt/00014.png', 'aixi_shape_256_texture/train/gt/00015.png', 'aixi_shape_256_texture/train/gt/00016.png', 'aixi_shape_256_texture/train/gt/00017.png', 'aixi_shape_256_texture/train/gt/00018.png', 'aixi_shape_256_texture/train/gt/00019.png', 'aixi_shape_256_texture/train/gt/00020.png', 'aixi_shape_256_texture/train/gt/00021.png', 'aixi_sha

In [84]:
import os

path_test = "aixi_shape_256_texture/val/"## TODO: posar el vostre path

files = os.listdir(path_train)
img_files2 = list([f"{path_train}{p}" for p in files if p.endswith('.png')])
label_files2 = list([f"{path_train}gt/{p}" for p in files if p.endswith('.png')])

## TODO: Comprovar que img_files i label_files tenen la informació del path de cada imatge

#### Crear una classe que ens permeti obtenir una tupla (imatge, etiqueta)

Aquesta classe hereta de la superclasse _Dataset_ i com a mínim ha de tenir els mètodes:

1. `__len__(self)`: retorna la longitud del dataset
2. `__getitem__(self, index)`: retorna l'element que es troba a la posició marcada pel valor d'index. Quan parlam d'un element parlam de la imatge i de la seva etiqueta.

El constructor i els atributs de la classe els he decidit jo:

- Llista amb els _paths_ a les imatges
- Llista amb els _paths_ a les imatges de gt que ens serviràn per calcular l'etiqueta de la imatge
- Un objecte transform

A la classe podeu afegir tants mètodes públics i privats com necessiteu

In [85]:

# Constructor del dataset.
class AIXI_Shape(Dataset):
    def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform

    def __len__(self): 
        return self.len

    def __getitem__(self, index):
        path = self.paths[index]
        image =cv2.imread(path,  cv2.IMREAD_GRAYSCALE) ## llegir la imatge que és al path
        image = self.transform(image)
        
        image2 = cv2.imread(self.labels[index])
        sum_channels = cv2.add(image2[:, :, 0], cv2.add(image2[:, :, 1], image2[:, :, 2]))
        contours,_  = cv2.findContours(sum_channels, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return (image, len(contours))

# image normalization
transform = transforms.Compose([
    transforms.ToTensor()
    ## Si voleu posar alguna cosa aquí, és feina vostra
])

# creació dels conjunts d'entrenament i test
train_ds = AIXI_Shape(img_files, label_files, transform)
test_ds = AIXI_Shape(img_files2, label_files2, transform)
# El test l'heu de crear vosaltres
train_dl = DataLoader(train_ds, batch_size=64)
test_dl = DataLoader(test_ds, batch_size=64)


## Xarxa
Com sempre, vosaltres us encarregau de dissenyar la xarxa:

In [90]:
class MyNet(nn.Module):

    def __init__(self):
        super(MyNet, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv_2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_3 = nn.Conv2d(in_channels=12, out_channels=1, kernel_size=3, stride=1)
        self.max_pool2d_2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        #TODO definir la xarxa.

        # Ajuda: A la darrera capa convolucional estaría molt bé que: out_channels = al nombre de sortides de la xarxa
           

    def forward(self, x):
        
        # TODO
        y = self.conv_1(x)
        y = self.conv_2(y)
        y = self.max_pool2d(y)
        y = self.conv_3(y)
        y = self.max_pool2d_2(y)
        y = self.relu(y)
        # Ajuda: no hi pot haver nombre de cel·lules negatius ;)
        return y.squeeze() # Aquesta funció en servirà per eliminar dimensions de mida 1.

# Entrenament

El blucle d'entrenament és el de sempre. Només heu de pensar quina funció de pèrdua heu d'emprar per el vostre/nostre problema

In [91]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data.float())
        print(output.shape)
        print(target.shape)
        
      
        loss = F.mse_loss(output,target)
        loss.backward()
        optimizer.step()
      
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            
            test_loss += F.mse_loss(output,target)#TODO
          
   
    test_loss /= len(test_loader.dataset)

    
    return test_loss

## Entrenament

In [92]:
use_cuda = False
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

epochs = 3# TODO
lr = 0.001#TODO

model = MyNet().to(device)


optimizer = optim.Adam(model.parameters(),lr=lr)# TODO

# Guardam el valor de pèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_dl, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_dl)


torch.Size([64, 63, 63])
torch.Size([64])


C:\Users\Jaume\AppData\Local\Temp\ipykernel_7360\3948970475.py:16: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 63, 63])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output,target)


RuntimeError: The size of tensor a (63) must match the size of tensor b (64) at non-singleton dimension 2

## Validació

Heu de fer vosaltres la validació depenent del problema que voldreu resoldre